# Custom Loss Function

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.losses import Loss

2023-12-07 15:54:40.018663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Custom Loss Function - Huber Loss 

### Prep Dataset

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

### Train Model

In [3]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)


print(model.predict([10.0]))

1/1 [==============================] - 0s 89ms/step
[[18.982853]]


### Custom Loss

In [4]:
def my_huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [5]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss)
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

1/1 [==============================] - 0s 49ms/step
[[18.781265]]


### Custom Loss with params

In [6]:
def my_huber_loss_with_threshold(threshold = 1):

    def my_huber_loss(y_true, y_pred):

        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)
    
    return my_huber_loss

In [7]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(1))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

1/1 [==============================] - 0s 56ms/step
[[18.59293]]


### Loss Functions to Classes

In [8]:
class CustomHuberLoss(Loss):

    threshold = 1

    def __init__(self, threshold):
        
        super().__init__()
        self.threshold = threshold

    def call(self, y_true, y_pred):

        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold

        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [9]:
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=CustomHuberLoss(1))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

1/1 [==============================] - 0s 55ms/step
[[18.79461]]
